# HSE Summer School NLP&DA

![](https://www.hse.ru/data/2014/06/25/1309038576/logo_hse_cmyk_e.jpg)

Учебный проект по теме "Тональность отношений субъектов (именованных сущностей)", предполагающий разработку участниками школы под руководством тьюторов программных средств, решающих задачу определения мнения сторон о различных событиях, освещаемых в новостях

Нужен алгоритм, который по выделенным сущностям может найти пары этих сущностей в тексте и вырезать соответствующие куски между ними + несколько слов справа-слева. Возможно, имеет смысл идти от N-ой выделенной сущности до N+2, и включать всё до неё. 

Чем может помочь синтаксический анализ?

Чем поможет морфологический анализ?

In [220]:
import sys 
sys.path.append("/Users/dmitrys/anaconda2/lib/python2.7/site-packages/")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pymorphy2
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()

import string
%matplotlib inline

## Utility functions

In [325]:
def loadAnswer(number):
    with open("Texts/art{}.opin.txt".format(number)) as f:
        d = f.read()
    return d

def loadText(number):
    with open("Texts/art{}.txt".format(number)) as f:
        d = f.read()
    return d

def transformAnnotation(number):
    """
    Given number loads txt file with annotation 
    Returns DataFrame with transformed annotation
    """
    with open("Texts/art{}.ann".format(number)) as f:
        d = f.read()

    d = d.split("\n")

    for i in range(len(d)):
        d[i] = d[i].split("\t")

    d = pd.DataFrame(d)
    d.drop([0], axis=1, inplace=True)
    d = pd.concat([d, pd.DataFrame(d[1].apply(lambda x: x.split()).tolist())], axis=1)
    d.columns = ["to_delete", "entity", "entity_car", "pos_1", "pos_2"]
    d.drop(["to_delete"], axis=1, inplace=True)
    d["entity"] = d["entity"].apply(lambda x: x.strip("\r"))
    d["entity"] = d["entity"].apply(lambda x: x.decode("utf8"))
    
    return d

def transformAnswer(number):
    """
    Given number loads txt file with answer 
    Returns DataFrame with transformed answer
    """
    answ = loadAnswer(number)
    answ = answ.split("\n")

    for i in range(len(answ)):
        answ[i] = answ[i].split(",")

    answ = pd.DataFrame(answ)
    answ.columns = ["entity_1", "entity_2", "attitude", "time"]
    answ.dropna(inplace=True)
    answ.time = answ.time.apply(lambda x: x.strip("\r"))
    
    return answ

def loadRuSentiLex():
    """
    Loads the RuSentiLex 2017 dictionary
    Returns data frame with ["word", "tag", "word_lemmatized", "tone", "certainty"]
    """
    with open("RuSentiLex2017_revised_2.txt") as f:
        Rusentilex = f.read().decode("cp1251").encode("utf8")
        Rusentilex = Rusentilex[1510:]

    Rusentilex = Rusentilex.split("\n")
    for i, item in enumerate(Rusentilex):
        Rusentilex[i] = item.split(",")

    for i in Rusentilex:
        if len(i) < 5:
            Rusentilex.remove(i)

    Rusentilex = pd.DataFrame(Rusentilex)
    Rusentilex.drop([5, 6, 7], axis=1, inplace=True)
    Rusentilex.columns = ["word", "tag", "word_lemmatized", "tone", "certainty"]
    Rusentilex["certainty"] = Rusentilex["certainty"].apply(lambda x: x.strip('\r'))
    
    
    Rusentilex["tone"] = Rusentilex["tone"].apply(lambda x: x.strip(' '))
    Rusentilex["certainty"] = Rusentilex["certainty"].apply(lambda x: x.strip(' '))
    
    return Rusentilex

def cleanString(myString):
    return myString.translate(None, string.punctuation).decode('utf-8')

![](http://cathyreisenwitz.com/wp-content/uploads/2016/01/no.jpg)

Словарь РуСентиЛекс

Структура: 
- 1 слово или словосочетание,
- 2 Часть речи или синтаксический тип группы,
- 3 слово или словосочетание в лемматизированной форме, 
- 4 Тональность: позитивная (positive), негативная(negative), нейтральная (neutral) или неопределеная оценка, зависит от контекста (positive/negative),
- 5 Источник: оценка (opinion), чувство (feeling), факт (fact),
- 6 Если тональность отличается для разных значений многозначного слова, то перечисляются все значения слова по тезаурусу РуТез и дается отсылка на сооветствующее понятие - имя понятия в кавычках.

In [149]:
Rusentilex = loadRuSentiLex()

In [150]:
Rusentilex.head()

,word,tag,word_lemmatized,tone,certainty
0,аборт,Noun,аборт,negative,fact
1,абортивный,Adj,абортивный,negative,fact
2,абракадабра,Noun,абракадабра,negative,opinion
3,абсурд,Noun,абсурд,negative,opinion
4,абсурдность,Noun,абсурдность,negative,opinion


In [151]:
Rusentilex.certainty.value_counts()

opinion             9269
fact                5297
feeling             1536
operator              41
negative               2
positive               2
fact В ТЕЗ empty       1
fact В ТЕЗ EMPTY       1
opinegativenion        1
Name: certainty, dtype: int64

In [169]:
def getSentimentCertainity(word):
    tone, certainty =  Rusentilex[["tone", "certainty"]][Rusentilex.word.isin([word])].values[0]
    return tone, certainty

In [130]:
getSentimentCertainity("аборт")

array(['negative', 'fact'], dtype=object)

In [326]:
entities = transformAnnotation(2)
a = transformAnswer(2) 

In [171]:
# надо идти по всем сущностям с правилами остановки

In [172]:
t = loadText(2)

In [175]:
print(t[:500])

{Author, Unknown} 

{Author, Unknown} Россия и кратковременный успех в Сирии

{Author, Unknown} inosmi.ruПосмотреть оригиналапрель 5-го, 2016

{Author, Unknown} Россия реализовала свои цели в Сирии, по крайней мере в краткосрочной перспективе. 
{Author, Unknown} Что будет дальше, пока неясно. 
{Author, Unknown} Точно можно сказа


In [161]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [168]:
p = morph.parse(u"аборт")[0]
p.tag.case

u'nomn'

In [259]:
entities[~entities.entity.isin(["Unknown", "Author"])].head(10)

,entity,entity_car,pos_1,pos_2
4,Россия,LOC,39,45
5,Сирии,LOC,72,77
10,Россия,LOC,164,170
11,Сирии,LOC,195,200
18,Москвы,LOC,521,527
19,Сирии,LOC,540,545
20,России,LOC,584,590
23,Иран,LOC,728,732
24,Башара Асада,PER,773,785
25,Кремля,ORG,815,821


In [255]:
def Text_to_dict(number):
    t = loadText(number)
    text_dict = t.split("\n\n")
    text_dict = {i:parag for i, parag in enumerate(text_dict)}
    sentence_dict = {i:{} for i in range(len(text_dict))}

    for key, paragraph in text_dict.iteritems():
        paragraph = paragraph.split("{Author, Unknown}")
        for sent_numbet, sentence in enumerate(paragraph):

            sentence = cleanString(sentence)
            if len(sentence) != 0:
                sentence_dict[key][sent_numbet] = tknzr.tokenize(sentence)
    
    return sentence_dict

In [261]:
text = Text_to_dict(2)

In [263]:
pprint(text)

{0: {1: []},
 1: {1: [u'\u0420\u043e\u0441\u0441\u0438\u044f',
         u'\u0438',
         u'\u043a\u0440\u0430\u0442\u043a\u043e\u0432\u0440\u0435\u043c\u0435\u043d\u043d\u044b\u0439',
         u'\u0443\u0441\u043f\u0435\u0445',
         u'\u0432',
         u'\u0421\u0438\u0440\u0438\u0438']},
 2: {1: [u'inosmiru',
         u'\u041f\u043e\u0441\u043c\u043e\u0442\u0440\u0435\u0442\u044c',
         u'\u043e\u0440\u0438\u0433\u0438\u043d\u0430\u043b\u0430\u043f\u0440\u0435\u043b\u044c',
         u'5\u0433\u043e',
         u'2016']},
 3: {1: [u'\u0420\u043e\u0441\u0441\u0438\u044f',
         u'\u0440\u0435\u0430\u043b\u0438\u0437\u043e\u0432\u0430\u043b\u0430',
         u'\u0441\u0432\u043e\u0438',
         u'\u0446\u0435\u043b\u0438',
         u'\u0432',
         u'\u0421\u0438\u0440\u0438\u0438',
         u'\u043f\u043e',
         u'\u043a\u0440\u0430\u0439\u043d\u0435\u0439',
         u'\u043c\u0435\u0440\u0435',
         u'\u0432',
         u'\u043a\u0440\u0430\u0442\u043a\u043e\u044

In [358]:
# for key, value in t.iteritems():
#     print("paragraph number", key)
#     for another_key, another_value in value.iteritems():
        
#         print("sentence_number", another_key)
#         for word in another_value:
#             print word

In [329]:
entities = entities[~entities.entity.isin(["Unknown", "Author"])].entity.reset_index(drop=True)

In [360]:
def getEntityPositions(text, entities):
    ENTITIES = pd.DataFrame(columns=["entity", "paragraph", "sentence", "word"])
    for paragraph in text:
        for sentence in text[paragraph]:
            for word in text[paragraph][sentence]:
                if word in list(entities[entity_key:]):
                    ENTITIES = ENTITIES.append({"entity":word, 
                                               "paragraph":paragraph, 
                                               "sentence":sentence, 
                                               "word":text[paragraph][sentence].index(word)},
                                              ignore_index = True)
    return ENTITIES

In [361]:
ENTITIES = getEntityPositions(text, entities)

In [362]:
ENTITIES

,entity,paragraph,sentence,word
0,Россия,1,1,0
1,Сирии,1,1,5
2,Россия,3,1,0
3,Сирии,3,1,5
4,Москвы,4,1,2
5,Сирии,4,1,5
6,России,4,1,11
7,Иран,5,1,0
8,Асада,5,1,8
9,Кремля,5,1,13


In [ ]:
"Россия"

In [339]:
u"Россия" == entities[entity_key:][0]

True

In [330]:
u"Россия" == entities[0]

True

In [317]:
entities[entity_key:].apply(lambda x: x.decode("utf8"))

0                   Россия
1                    Сирии
2                   Россия
3                    Сирии
4                   Москвы
5                    Сирии
6                   России
7                     Иран
8             Башара Асада
9                   Кремля
10                  Россия
11                   Ирана
12                  России
13                 Дамаске
14                     США
15            Барака Обамы
16                   Сирии
17                  Россию
18                    ИГИЛ
19              Башар Асад
20              Хафез Асад
21                   Сирии
22                   Сирии
23      Владимиром Путиным
24    Биньямином Нетаньяху
25                 Израиль
26        Голанские высоты
27                  Москвы
28                   Сирии
29                  Москва
              ...         
56                 Россией
57                Израилем
58                   Ирана
59                   Асада
60                 Дамаске
61                Тегерана
6

In [297]:
paragraph_key = 0
sentence_key = 0
word_index = 0

x = 0

for entity_number in range(len(entities)):
    entity = entities[entity_number]
    for paragraph_number, sentence in text.iteritems():
        for sentence_number, word_list in sentence.iteritems():
            try:
                entity_index = word_list.index(entity.decode("utf8"))
            except:
                x += 1
            

На абзацы можно разбивать по двойным \n\n

In [110]:
print(t[:1000].split("\n\n")[4])

{Author, Unknown} Что касается Москвы, то будущее Сирии пока слабо представляется без участия России, особенно если принять 


In [34]:
print(t.split("{Author, Unknown}")[6])

 Точно можно сказать одно: подавляющее большинство сирийских граждан не смогут забыть российского участия в войне, которая была направлена против всего сирийского народа.





In [20]:
from pymystem3 import Mystem

m = Mystem()
def lemmatize(text, mystem=m):
    try:
        return "".join(m.lemmatize(text)).strip()  
    except:
        return " "

In [15]:
for i in range(10):
    print(lemmatize(t[i]))

{Author,
Unknown}
{Author,
Unknown}
россия
и
кратковременный
успех
в
сирия
